In [4]:
import json

In [7]:
!pip install pypdf2

In [5]:
from PyPDF2 import PdfReader

In [6]:
def text_extractor(path: str):
    reader = PdfReader(path)
    # page = reader.pages[0]
    return reader.pages
    # return page.extract_text()

### Proceso para extraer el diccionario de datos del archivo PDF y convertirlo en un archivo de tipo json

In [15]:
def process_data_dictionary(pages):
    data = []
    for page in pages:
        txt = page.extract_text()
        ntext = txt.split("\n") # Split text by jumplines
        for i, t in enumerate(ntext):
            if "Valo r" in t:
                text = ntext[i+2].split()
                if len(text) > 1 and "Etiqueta" in text:
                    nquestion = ntext[i+1].split()[1].strip() # Position of question. Eg. Posición 5
                    question = " ".join(text[2:]).strip() # Text question
                    qtype = ntext[i+3] # Question type
                    if "Tipo" not in qtype: # The question continue
                        question += " " + "".join(ntext[i+3]).strip()
                        qtype = ntext[i+4].split()[1] if "Tipo" in ntext[i+4] else None # Updating Question type
                        rtype = ntext[i+6].split()[1] if "Medición" in ntext[i+6] else None
                        n_value = i+7 # Next Value
                    else:
                        qtype = qtype.split()[1] # Removing 'Tipo'
                        rtype = ntext[i+5].split()[1] if "Medición" in ntext[i+5] else None
                        n_value = i+6 # Next Value
                    # Get answers options 
                    options = {}
                    try:
                        while "Valo" not in ntext[n_value] and rtype:
                            opt = ntext[n_value].split()
                            options[opt[0].strip()] = " ".join(opt[1:])
                            n_value += 1
                    except Exception as e: # Ignore 2 questions (implies much effort to fix)
                        print(f"Exception: {e}\nNLINE: {n_value}\nText:{t}\n")
                    data.append({
                        "nq": nquestion,
                        "question": question,
                        "qtype": qtype,
                        "rtype": rtype,
                        "options": options

                    })
                    n_value = 0
    # Serializing json
    json_object = json.dumps(data, indent=4, ensure_ascii=False)
        
    # Writing to json file
    with open("data_dict_e_nal2.json", "w", encoding="utf-8") as outfile:
        outfile.write(json_object)

In [17]:
path = "diccionario20de20codigos.pdf"
pages = text_extractor(path)
# print(pages[0].extract_text())

In [18]:
print("Generating json..")
process_data_dictionary(pages)

Generating json..
TEXTT  ['Valo r', 'Posición 2', 'Etiqueta 1. ¿Me podría decir su edad por favor?', 'Tipo Numérico', 'Formato F2', 'Medición Escala', 'Valo r', 'Posición 3', 'Etiqueta 2. ¿Usted ha navegado por Internet o se ha conectado a alguna r ed social a través de ', 'Internet?', 'Tipo Numérico', 'Formato F1', 'Medición Nominal', '1 Si', '2 No', 'Valo r', 'Posición 4', 'Etiqueta 3. ¿Cuándo fue la última vez que navegó en Internet o estuvo co nectado a las redes ', 'sociales?', 'Tipo Numérico', 'Formato F1', 'Medición Nominal', '1 Hace 7 días o menos', '2 Entre 8 a 15 días', '3 Entre 16 a 30 días', '4 Hace más de 30 días', 'Valo r', 'Posición 5', 'Etiqueta 1a. ¿Cuáles son las razones por las cuales no se conecta a Inte rnet regularmente?', 'Tipo Numérico', 'Formato F1', 'Medición Nominal', '1 No existe el servicio en el barrio/localidad', '2 El costo del servicio alto', '3 No tiene equipos para conectarse', '4 No sabe qué es Internet', '5 No sabe cómo usar Internet', '6 No está in